## 표지판 예측 모델 학습

In [87]:
# 파일 열고 URL주소 뒤에 [ ?kernel_name=py374 ] <- 붙여서 실행

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import tensorflow as tf
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, Conv2DTranspose, LeakyReLU, UpSampling2D
from keras import optimizers
# 
from keras.layers.normalization import BatchNormalization as BN

from keras.layers import Lambda, Reshape, Add, AveragePooling2D, MaxPooling2D, Concatenate, SeparableConv2D
from keras.models import Model
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

from keras.callbacks import ModelCheckpoint

from keras.regularizers import l2

from keras.preprocessing.image import array_to_img, img_to_array, load_img

from sklearn.model_selection import train_test_split

from PIL import Image, ImageDraw, ImageFilter

In [88]:
import keras
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
sess  = tf.Session(config=config) 
keras.backend.set_session(sess)

In [89]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 데이터세트 만들기

In [90]:
dir_url = './data/train/'

print( 'dir_url : {}'.format(dir_url) )
files = os.listdir(dir_url)
i = 1

print("[INFO] loading images...")
LABELS = set( ["30","50","60","NO_U-turn","NO_left","NO_parking_stop","NO_right"] )
data = []
labels = []
train_path = './data/train'

for files_list in files : 
    print(files_list) # 30, 50, 60
    file = os.listdir(dir_url+files_list)
    for image in os.listdir(dir_url+files_list):
        imagepath = train_path + '/' + files_list + '/' + image
        
        img = cv2.imread(imagepath)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(64,64) )
        data.append(img)
        
        if files_list not in LABELS:
            continue
        labels.append(files_list)
        

# convert the data and labels to NumPy arrays
print("[INFO] End loading images...")
data = np.array(data)
labels = np.array(labels)

data = data / 255.0 

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

dir_url : ./data/train/
[INFO] loading images...
NO_U-turn
NO_left
NO_parking_stop
50
60
NO_right
30
[INFO] End loading images...


In [91]:
# 데이터 셋 크기 확인
data.shape

(3018, 64, 64, 3)

In [92]:
# 레이블 확인
lb.classes_

array(['30', '50', '60', 'NO_U-turn', 'NO_left', 'NO_parking_stop',
       'NO_right'], dtype='<U15')

In [93]:
# 레이블 길이 확인
len(lb.classes_)

7

In [94]:
labels.shape

(3018, 7)

In [95]:
# 데이터 분리 - train / test 용
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

## 모델 생성

- 컨볼루션 레이어 : 입력 이미지 크기 64 x 64, 입력 이미지 채널 3개, 필터 크기 3 x 3, 필터 수 32개, 활성화 함수 ‘relu’
- 맥스풀링 레이어 : 풀 크기 2 x 2
- 드롭 레이어
- 컨볼루션 레이어 : 필터 크기 3 x 3, 필터 수 64개, 활성화 함수 ‘relu’
- 맥스풀링 레이어 : 풀 크기 2 x 2
- 드롭 레이어
- 플래튼 레이어
- 덴스 레이어 : 출력 뉴런 수 128개, 활성화 함수 ‘relu’
- 드롭 레이어
- 덴스 레이어 : 출력 뉴런 수 3개, 활성화 함수 ‘softmax’

In [96]:
from keras.models import Sequential

In [97]:
model = Sequential()
model.add( Conv2D( 32, kernel_size=(3,3), activation='relu', input_shape=(64,64,3) ) )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )

model.add( Conv2D( 64, kernel_size=(3,3), activation='relu') )
model.add( MaxPooling2D( pool_size=(2,2) ) )
model.add( Dropout(0.25) )

model.add( Flatten() )

model.add( Dense(128, activation='relu') )
model.add( Dropout(0.25) )

model.add( Dense( len(lb.classes_), activation='softmax' ) )

In [98]:
# 모델 구성 확인
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 12544)             0         
__________

#### 모델 컴파일
- [손실함수] 
- categorical_crossentropy 를 사용

- [최적화 함수]
- 1. 아담 
    -> opt = Adam(lr=1e-3, decay=1e-3 / 50)
- 2. SGD 
    -> opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
- 3. rmsprop

In [99]:
print("[INFO] compiling model...")
model.compile(loss="categorical_crossentropy", optimizer='rmsprop', metrics=['accuracy'] )

[INFO] compiling model...


In [100]:
start = time.time()

# GPU 로 돌리기 (with 포함시킬것)
with tf.device('/device:GPU:0'):
    model.fit( trainX, trainY, batch_size=128, epochs= 50 , validation_split= 0.2 )
    
elapsed_time = time.time() - start
print ("elapsed_time:{}".format(elapsed_time) + "[sec]")

Train on 1810 samples, validate on 453 samples
Epoch 1/50
1810/1810 [==============================] - 1s 428us/step - loss: 1.9242 - acc: 0.3945 - val_loss: 1.2912 - val_acc: 0.5121
Epoch 2/50
1810/1810 [==============================] - 0s 230us/step - loss: 1.3023 - acc: 0.4994 - val_loss: 1.6340 - val_acc: 0.5342
Epoch 3/50
1810/1810 [==============================] - 0s 275us/step - loss: 1.1735 - acc: 0.5271 - val_loss: 1.1063 - val_acc: 0.6358
Epoch 4/50
1810/1810 [==============================] - 0s 255us/step - loss: 0.9808 - acc: 0.6227 - val_loss: 1.0951 - val_acc: 0.6313
Epoch 5/50
1810/1810 [==============================] - 0s 265us/step - loss: 0.8649 - acc: 0.6481 - val_loss: 0.7549 - val_acc: 0.6887
Epoch 6/50
1810/1810 [==============================] - 0s 242us/step - loss: 0.7426 - acc: 0.7044 - val_loss: 0.7783 - val_acc: 0.7373
Epoch 7/50
1810/1810 [==============================] - 0s 246us/step - loss: 0.6353 - acc: 0.7613 - val_loss: 0.8116 - val_acc: 0.6137
E

In [101]:
# 모델 예측
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1), target_names=lb.classes_) )

[INFO] evaluating network...
                 precision    recall  f1-score   support

             30       0.99      0.92      0.95       183
             50       0.95      0.99      0.97       381
             60       0.97      0.96      0.97       120
      NO_U-turn       1.00      0.95      0.98        21
        NO_left       1.00      1.00      1.00        30
NO_parking_stop       1.00      0.93      0.97        15
       NO_right       1.00      1.00      1.00         5

       accuracy                           0.97       755
      macro avg       0.99      0.96      0.98       755
   weighted avg       0.97      0.97      0.97       755



In [102]:
#모델을 JSON 파일 형식으로 만들어 저장하기
model_json = model.to_json()
with open("./model/model.json", "w") as json_file : 
    json_file.write(model_json)

In [103]:
# 모델 가중치(weights) 저장
model.save_weights('./model/model.h5')

## 학습된 모델을 사용하여 ROI영역 레이블 예측 및 그리기

In [104]:
# import the necessary packages
from keras.models import load_model
from collections import deque
import numpy as np
import argparse
import pickle
import cv2

In [105]:
# 독일 이미지로 테스트

In [106]:
# 7가지 이미지 데이터 예측
test_path_30 = './test_image/30_black.jpg'                             # 성공
test_path_50 = './test_image/50_29.jpg'                                # 성공
test_path_60 = './test_image/60_211.jpg'                              # 성공
test_path_return = './test_image/test_no_return2.png'            # 성공
test_path_no_left = './test_image/test_no_left.png'                # 성공
test_path_no_right= './test_image/test_no_right.png'             # 실패 -> 배경이 검은색이라 예측실패인 걸로 예측됨
test_path_no_right2= './test_image/test_no_parking.jpg'        # 성공
test_path_no_parking = './test_image/test_no_parking.jpeg'   # 성공

# 예측 테스트
frame = cv2.imread(test_path_no_right2)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.resize(frame, (64, 64)).astype("float32")

# 예측
preds = model.predict(np.expand_dims(frame, axis=0))[0]
print(preds)

# 레이블 표시
label = lb.classes_[np.argmax(preds)]
text = "label text: {}".format(label)
print(text)

[0. 0. 0. 0. 0. 0. 1.]
label text: NO_right


In [107]:
## ---------------------------------------------------------------------------------연경아 부탁해---------------------------------------------------------------------------------- ##
# 전처리 과정을 거친 우리 데이터로 예측 테스트
img = cv2.imread('./test_image/sample3_515.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3, 3), 0)
circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1.5, 30, None, 570)

#print(circles) # (x, y, 반지름)
if circles is not None :
    circles = np.uint16(np.around(circles))
    for i in circles[0, :] :
        x = i[0]
        y = i[1]
        r = i[2]
        #print(x, y, r)
        #print((x-r, y-r), (x+r, y+r))
        cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
        cv2.circle(img, (i[0], i[1]), 2, (0, 0, 255), 5)
        cv2.rectangle(img, (x-r, y-r), (x+r, y+r), (255, 0, 0), 1)
      #x,y 원의 중심 좌표 / r : 반지름  
frame = img[y-r:y+r, x-r:x+r]
## -------------------------------------------------------------------------------------화이팅-------------------------------------------------------------------------------------------- ##


frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
frame = cv2.resize(frame, (64, 64)).astype("float32")

# 예측 후 , roi 영역 표시
preds = model.predict(np.expand_dims(frame, axis=0))[0]
print('label : {}'.format(preds))

#  레이블 확인
label = lb.classes_[np.argmax(preds)]
text = "predict: {}".format(label)
print(text)
cv2.putText(img, text, (x-50, y-50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)

# 이미지 확인
cv2.imshow('final', img)

# 종료 : 키보드 클릭
cv2.waitKey()
cv2.destroyAllWindows()

label : [0. 1. 0. 0. 0. 0. 0.]
predict: 50


## 영상파일 예측 텍스트 첨부

In [108]:
# 모델 불러오기

In [109]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from keras.models import load_model
import numpy as np
import pickle
import cv2
#저장된 JSON 파일로 부터 모델 로드하기
from keras.models import model_from_json

In [110]:
json_file = open("./model/model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)


In [111]:
#로드한 모델에 Weight 로드하기
loaded_model.load_weights("./model/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [112]:
#모델 컴파일 후 Evaluation
loaded_model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=['accuracy'])

In [113]:
import imutils

In [114]:
# 영상 파일 경로
video_path = './video/video1.mp4'
cap = cv2.VideoCapture(video_path)

In [115]:
#  비디오 writer와 프레임 크기 초기화
writer = None
(W, H) = (None,None)

In [116]:
# 전체 비디오 처리에 걸리는 시간을 추정하기 위해 비디오파일의 총 프레임 수를 결정
try : 
    prop = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
            else cv2.CAP_PROP_FRAME_COUNT
    total = int( cap.get(prop) )
    print("{} total frames in video".format(total) )
except :
    print("could not determine # of frames in video")
    print("no approx. completion time can be provided")
    total = -1

10161 total frames in video


In [117]:
video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]
print(video_file)
print(video_name)

video1.mp4
video1


In [ ]:
if cap.isOpened() :
    fps = cap.get(cv2.CAP_PROP_FPS)
    delay = int(1000/fps)
    print(fps)    # 30.0028139633606
    print(delay) # 33

## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 
    while True :
        ret, img = cap.read()                      # 다음 프레임 읽기
        
        if ret :                                           # 프레임 읽기 정상 
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 
            img2 = img.copy()
            (H, W) = img.shape[:2]
           
            # HSV 영상으로 변환
            hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
            img2 = cv2.bilateralFilter(img, 9, 105, 105)
            r, g, b = cv2.split(img2)
            equalize1= cv2.equalizeHist(r)
            equalize2= cv2.equalizeHist(g)
            equalize3= cv2.equalizeHist(b)
            equalize = cv2.merge((equalize1, equalize2, equalize3))
            img2 = equalize

            # 색상별 영역 지정
            red1 = np.array([0, 50, 50])
            red2 = np.array([15, 255, 255])
            red3 = np.array([165, 50, 50])
            red4 = np.array([180, 255, 255])

            # 색상에 따른 마스크 생성
            mask_red1 = cv2.inRange(hsv, red1, red2)
            mask_red2 = cv2.inRange(hsv, red3, red4)
            mask_red = mask_red1 + mask_red2

            numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red2)
            
            
            for idx, centroid in enumerate(centroids) :
                if stats[idx][0] == 0 and stats[idx][1] == 0 :
                    continue

                if np.any(np.isnan(centroid)) :
                    continue

                x, y, W, H, area = stats[idx]                                       # x,y -> 좌표 / W,H -> 너비 , 높이
                centerX, centerY = int(centroid[0]), int(centroid[1])

                if area > 50 and abs( W - H ) < 5 : 
                    # 원 검출
                    detected_img          = img2[ y : y + H , x : x + W ]
                    gray_detected_img  = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                    gray_detected_img  = cv2.resize(gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5))
                    circles                    = cv2.HoughCircles(gray_detected_img,
                                                                              cv2.HOUGH_GRADIENT,
                                                                              1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)
                    
                    if circles is not None :
                        circles = np.uint16( np.around(circles) )
                        
                        for i in circles[0, : ] :
                            # 표지판 이미지 추출
                            frame = img[ y - 3 : y + H + 3 , x - 3 : x + W + 3 ]
                            try : 
                                # bgr -> rgb 로 배열순서 변경
                                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                                # 리사이즈 : 64 * 64
                                frame = cv2.resize(frame, (64, 64) )
                                # 예측
                                preds = model.predict(np.expand_dims(frame, axis=0))[0]

                                #  레이블 확인
                                label = lb.classes_[np.argmax(preds)]
                                text = "predict: {}".format(label)
                                
                                cv2.putText( img, text, ( x + 50, y + 50 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                                cv2.rectangle(img, (x - 3, y - 3), (x + W + 3, y + H + 3), ( 255, 0, 0 ), 1)
                            except :
                                continue
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##           
            cv2.imshow('video_file', img)        # 화면에 표시
            #cv2.waitKey(10)                     # fps에 맞게 시간 지연
            if cv2.waitKey(1) == 27 :
                cap.release()
                cv2.destroyAllWindows()
        else :
            break
else : # 영상 경로 확인
    print("can't open video.")

cap.release()
cv2.destroyAllWindows()

In [ ]:
# 영상 파일 경로
video_path = './video/video1.mp4'
cap = cv2.VideoCapture(video_path)

if cap.isOpened() :
    fps = cap.get(cv2.CAP_PROP_FPS)
    delay = int(1000/fps)
    #print(fps)    # 30.0028139633606
    #print(delay) # 33

## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 
    while True :
        ret, img = cap.read()                            # ret : 프레임 존재 유무/ img : 프레임 읽기( cap.read() )
        
        if not ret :                                           # 비디오의 마지막 프레임 확인  
            break
        
        if W is None or H is None:                    # 프레임의 크기가 잡히지 않는 경우 프레임 크기를 잡는다.
            (H, W) = img.shape[:2]

        img2 = img.copy()
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 
            

        # HSV 영상으로 변환
        hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
        img2 = cv2.bilateralFilter(img, 9, 105, 105)
        r, g, b = cv2.split(img2)
        equalize1= cv2.equalizeHist(r)
        equalize2= cv2.equalizeHist(g)
        equalize3= cv2.equalizeHist(b)
        equalize = cv2.merge((equalize1, equalize2, equalize3))
        img2 = equalize

        # 색상별 영역 지정
        red1 = np.array([0, 50, 50])
        red2 = np.array([15, 255, 255])
        red3 = np.array([165, 50, 50])
        red4 = np.array([180, 255, 255])

        # 색상에 따른 마스크 생성
        mask_red1 = cv2.inRange(hsv, red1, red2)
        mask_red2 = cv2.inRange(hsv, red3, red4)
        mask_red = mask_red1 + mask_red2

        numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red2)


        for idx, centroid in enumerate(centroids) :
            if stats[idx][0] == 0 and stats[idx][1] == 0 :
                continue

            if np.any(np.isnan(centroid)) :
                continue

            x, y, W, H, area = stats[idx]                                       # x,y -> 좌표 / W,H -> 너비 , 높이
            centerX, centerY = int(centroid[0]), int(centroid[1])

            if area > 50 and abs( W - H ) < 5 : 
                # 원 검출
                detected_img          = img2[ y : y + H , x : x + W ]
                gray_detected_img  = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
                gray_detected_img  = cv2.resize(gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5))
                circles                    = cv2.HoughCircles(gray_detected_img,
                                                                          cv2.HOUGH_GRADIENT,
                                                                          1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)

                if circles is not None :
                    circles = np.uint16( np.around(circles) )

                    for i in circles[0, : ] :
                        # 표지판 이미지 추출
                        frame = img[ y - 4 : y + H + 4 , x - 4 : x + W + 4 ]
                        #frame = img[ y  : y + H  , x  : x + W ]
                        try : 
                            # bgr -> rgb 로 배열순서 변경
                            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                            # 리사이즈 : 64 * 64
                            frame = cv2.resize(frame, (64, 64) )
                            # 예측
                            preds = model.predict(np.expand_dims(frame, axis=0))[0]

                            #  레이블 확인
                            label = lb.classes_[np.argmax(preds)]
                            text = "predict: {}".format(label)

                            cv2.putText( img, text, ( x + 50, y + 50 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                            cv2.rectangle(img, (x - 4, y - 4), (x + W + 4, y + H + 4), ( 255, 0, 0 ), 1)
                        except :
                            continue
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##           
        cv2.imshow('video_file', img)        # 화면에 표시

        if cv2.waitKey(1) == 27 :
            cap.release()
            cv2.destroyAllWindows()
                
                
else : # 영상 경로 확인
    print("can't open video.")

cap.release()
cv2.destroyAllWindows()

In [118]:
# 영상 파일 경로
video_path = './video/video1.mp4'
cap = cv2.VideoCapture(video_path)

while True :
    ret, img = cap.read()                            # ret : 프레임 존재 유무/ img : 프레임 읽기( cap.read() )

    if not ret :                                           # 비디오의 마지막 프레임 확인  
        print('can`t open video')
        break

    if W is None or H is None:                    # 프레임의 크기가 잡히지 않는 경우 프레임 크기를 잡는다.
        (H, W) = img.shape[:2]

    img2 = img.copy()
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ## 

    # HSV 영상으로 변환
    hsv = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
    img2 = cv2.bilateralFilter(img, 9, 105, 105)
    r, g, b = cv2.split(img2)
    equalize1= cv2.equalizeHist(r)
    equalize2= cv2.equalizeHist(g)
    equalize3= cv2.equalizeHist(b)
    equalize = cv2.merge((equalize1, equalize2, equalize3))
    img2 = equalize

    # 색상별 영역 지정
    red1 = np.array([0, 50, 50])
    red2 = np.array([15, 255, 255])
    red3 = np.array([165, 50, 50])
    red4 = np.array([180, 255, 255])

    # 색상에 따른 마스크 생성
    mask_red1 = cv2.inRange(hsv, red1, red2)
    mask_red2 = cv2.inRange(hsv, red3, red4)
    mask_red = mask_red1 + mask_red2

    numOfLabels, img_label, stats, centroids = cv2.connectedComponentsWithStats(mask_red2)


    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue

        if np.any(np.isnan(centroid)) :
            continue

        x, y, W, H, area = stats[idx]                                       # x,y -> 좌표 / W,H -> 너비 , 높이
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 50 and abs( W - H ) < 5 : 
            # 원 검출
            detected_img          = img2[ y : y + H , x : x + W ]
            gray_detected_img  = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)
            gray_detected_img  = cv2.resize(gray_detected_img, (gray_detected_img.shape[1]*5, gray_detected_img.shape[0]*5))
            circles                    = cv2.HoughCircles(gray_detected_img,
                                                                      cv2.HOUGH_GRADIENT,
                                                                      1, 100, param1=50, param2=30, minRadius=10, maxRadius=0)

            if circles is not None :
                circles = np.uint16( np.around(circles) )

                for i in circles[0, : ] :
                    # 표지판 이미지 추출
                    frame = img[ y - 4 : y + H + 4 , x - 4 : x + W + 4 ]
                    #frame = img[ y  : y + H  , x  : x + W ]
                    try : 
                        # bgr -> rgb 로 배열순서 변경
                        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                        # 리사이즈 : 64 * 64
                        frame = cv2.resize(frame, (64, 64) )
                        # 예측
                        preds = model.predict(np.expand_dims(frame, axis=0))[0]

                        #  레이블 확인
                        label = lb.classes_[np.argmax(preds)]
                        text = "predict: {}".format(label)

                        cv2.putText( img, text, ( x + 80, y + 80 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 5)
                        cv2.rectangle(img, (x - 4, y - 4), (x + W + 4, y + H + 4), ( 255, 0, 0 ), 1)
                        #cv2.circle(frame, (i[0], i[1]), i[2], (0, 255, 0), 2)
                        #cv2.circle(frame, (i[0], i[1]), 2, (0, 0, 255), 5)
                    except :
                        continue
## ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- ##           
    cv2.imshow('video_file', img)        # 화면에 표시

    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()

error: OpenCV(3.4.7) /io/opencv/modules/highgui/src/window.cpp:358: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
